In [8]:
# Install required libraries
# !pip install yfinance pandas

"""
get_stock_data - 현재가 + 이동 평균선, 거래량 이동 평균선 + 365일, 180일, 90일, 30일 이내의 최고점, 최저점 및 역사상 신고가, 최저가
get_minute_stock_data - 1분봉 데이터 + 1분봉 이동 평균선

Date: 날짜 (YYYY-MM-DD 형식)
Close: 종가
Volume: 거래량
SMA_X: X일 단순 이동 평균선 (SMA)
VMA_X: X일 거래량 이동 평균선 (VMA)
"""

import yfinance as yf
import pandas as pd
import os
from datetime import datetime, timedelta

# Set up project directory
project_dir = 'data_csv'
data_dir = os.path.join(project_dir, '')
os.makedirs(data_dir, exist_ok=True)

def get_stock_data(ticker, start_date, end_date, interval='1d'):
    """
    주어진 주식 코드와 기간에 해당하는 주식 데이터를 받아오는 함수.

    Args:
        ticker (str): 주식 코드.
        start_date (str): 데이터의 시작 날짜 (YYYY-MM-DD 형식).
        end_date (str): 데이터의 종료 날짜 (YYYY-MM-DD 형식).
        interval (str): 데이터 간격 (1d, 1m 등).

    Returns:
        pd.DataFrame: 주식 데이터 프레임.
    """
    data = yf.download(ticker, start=start_date, end=end_date, interval=interval)
    data = data[['Close', 'Volume']]  # 종가 및 거래량 데이터 사용
    data = data.reset_index()
    data.columns = ['Date', 'Close', 'Volume']
    
    # 이동평균선 데이터 계산
    ma_columns = {}
    for ma in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60] + list(range(70, 710, 10)):
        ma_columns[f'SMA_{ma}'] = data['Close'].rolling(window=ma).mean()
        ma_columns[f'VMA_{ma}'] = data['Volume'].rolling(window=ma).mean()
    
    ma_df = pd.DataFrame(ma_columns)
    data = pd.concat([data, ma_df], axis=1)

    # 추가 데이터 계산
    data['365D_High'] = data['Close'].rolling(window=365).max()
    data['365D_Low'] = data['Close'].rolling(window=365).min()
    data['180D_High'] = data['Close'].rolling(window=180).max()
    data['180D_Low'] = data['Close'].rolling(window=180).min()
    data['90D_High'] = data['Close'].rolling(window=90).max()
    data['90D_Low'] = data['Close'].rolling(window=90).min()
    data['30D_High'] = data['Close'].rolling(window=30).max()
    data['30D_Low'] = data['Close'].rolling(window=30).min()
    data['AllTime_High'] = data['Close'].cummax()
    data['AllTime_Low'] = data['Close'].cummin()
    
    
    return data

def get_minute_stock_data(ticker, start_date, end_date):
    """
    1분봉 주식 데이터를 7일 단위로 받아오는 함수.

    Args:
        ticker (str): 주식 코드.
        start_date (str): 데이터의 시작 날짜 (YYYY-MM-DD 형식).
        end_date (str): 데이터의 종료 날짜 (YYYY-MM-DD 형식).

    Returns:
        pd.DataFrame: 주식 데이터 프레임.
    """
    all_data = []
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    delta = timedelta(days=7)

    while start < end:
        temp_end = min(start + delta, end)
        data = yf.download(ticker, start=start, end=temp_end, interval='1m')
        all_data.append(data)
        start = temp_end

    data = pd.concat(all_data)
    data = data[['Close', 'Volume']]  # 종가 및 거래량 데이터 사용
    data = data.reset_index()
    data.columns = ['Date', 'Close', 'Volume']
    
    # 이동평균선 데이터 계산
    ma_columns = {}
    for ma in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60] + list(range(70, 710, 10)):
        ma_columns[f'SMA_{ma}'] = data['Close'].rolling(window=ma).mean()
        ma_columns[f'VMA_{ma}'] = data['Volume'].rolling(window=ma).mean()
    
    ma_df = pd.DataFrame(ma_columns)
    data = pd.concat([data, ma_df], axis=1)
    
    return data

def save_data_to_csv(data, filename):
    """
    주어진 데이터를 CSV 파일로 저장하는 함수.

    Args:
        data (pd.DataFrame): 저장할 데이터 프레임.
        filename (str): 저장할 CSV 파일의 이름.
    """
    data.to_csv(filename, index=False)
    print(f'Data saved to {filename}')

# Example usage
ticker = '005930.KS'
start_date = '2004-01-01'
end_date = '2024-07-14'
filename = os.path.join(data_dir, 'samsung_stock_data.csv')

# 일봉 데이터 가져오기
data = get_stock_data(ticker, start_date, end_date, interval='1d')
save_data_to_csv(data, filename)

# 1분봉 데이터 가져오기 (최대 7일 범위 내)
minute_data = get_minute_stock_data(ticker, '2024-07-01', '2024-07-14')
minute_filename = os.path.join(data_dir, 'samsung_stock_minute_data.csv')
save_data_to_csv(minute_data, minute_filename)

# PER 데이터는 yfinance에서 직접 제공하지 않으므로 다른 소스를 통해 얻어야 합니다.
# 예를 들어, Yahoo Finance 웹사이트에서 직접 스크래핑하거나, 다른 금융 API를 통해 가져올 수 있습니다.


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Data saved to data_csv\samsung_stock_data.csv


Data saved to data_csv\samsung_stock_minute_data.csv
